# Review Summarization - Aspect based opinion mining

Summarize opinions of users about a product from a set of reviews. Extract the most common product features mentioned, most common opinion words used for a feature and the corresponding positive and negitive opinions about related to the feature and opinions. This way it becomes extremely easy to identify the most prominent positive and negitive features and opinions about a product. 

## Methodology:

1. Tokenize the reviews into sentences.

2. Create a set of nouns in each sentence. Extract the most common nouns in all of the reviews using apriori algorithm. This gives us the most frequent features/aspects of the product.

3. Classify each sentence containing a product feature as positive, negitive or neutral using the Liu and Hu opinion lexicon.

4. Extract opinion phrases from sentences using nltk chunking. We identify the most frequent opinion phrases using frequency distribution.

5. For each product feature and corresponding opinion phrase, we display the positive and negitive reviews.


## Sample:

**Product**: phone

**Features**: camera, battery, price, performance

**Opinion Phrases**: great camera, great battery, low light, heat issues

**Output for a feature**:

*Feature* - camera

*Positive* - great camera

*Reviews* - 
1. Superb slick phone, great camera nice depth mode like DSLR cameraphone is a beast n crazy fast...I am very happy with the purchase...
2. portrail photo is the awsome feature, perfect andoid product at hisAwesome product with great camera features.. Lighting fast processor...
3. Nice phone with great camera battery backup
4. Amazing phone in good price,with great camera experience

*Negitive* - low light

*Reviews* - 
1. low lighting photos could have been better.
2. Camera quality awesome on rear, but struggling on low lighting conditions a little bit as there sometimes appears noise in image captured in low light.


## Data:

We have a file containing 1000 reviews of One plus 5 from Amazon in json format.

*Each review has the following details attached with it:*
1.review_author 
2.review_header 
3.review_posted_date 
4.review_rating 
5.review_text

In [52]:
import nltk
import json
file_data=""
reviews_final_str=""
reviews_str=""
reviews_neg_str=""
with open('D:/reviews1_1000_data.txt','r',encoding="utf8",newline='') as f:
    file_data = json.load(f)

'''print sample data of first review'''
print(file_data[0][0])

{'review_author': ['Biplab'], 'review_header': ['Best value for money mid range killer'], 'review_posted_date': ['on 18 October 2017'], 'review_rating': ['5.0 out of 5 stars', '|', '|', '|'], 'review_text': ['Worth every penny. Best value for money mid range killer. Balance of power, storage, camera and battery.', 'Pros:', '1) Superb camera. Protrait mode is awesome (90/100). Good night light snaps.', '2) Blazing fast DashCharge. Charged 100% in almost 1 hour 20 minutes.', '3) Sound is loud and crystal clear.', '4) Oxygen OS is buttery smooth combined with SD835 and 6 GB of RAM.', '5) Regular updates', 'Cons:', 'One and only minor glitch is the lens flare effect in rear cams. But it is common in multi-lens mobile cams.', "Highly recommended to feel the premium segment's taste."]}


We tokenize all the reviews as sentences into sent_fullreview. Then the 1 and 2 star rated reviews are tokenized into sent_neg_review and the rest of the reviews into sent_review.

I have customized the PunktSentenceTokenizer to separate sentences on a few extra words and characters as well.

In [53]:
for r in file_data:
    for s in r:
        reviews_final_str = reviews_final_str + str(s['review_text'])
        if ((s['review_rating'][0][:1]=="1") or (s['review_rating'][0][:1]=="2")):
            reviews_neg_str = reviews_neg_str + str(s['review_text'])
        else:
            reviews_str = reviews_final_str + str(s['review_text'])
            
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktLanguageVars
'''We customize the ReviewLangVars class to separate sentences based on some additional keywords'''
class ReviewLangVars(PunktLanguageVars):
    sent_end_chars = ('pros:', 'cons:', '[','][','.','?','!')
    
    
sent_tokenizer1 = PunktSentenceTokenizer(lang_vars = ReviewLangVars())
sent_fullreview = sent_tokenizer1.tokenize(reviews_final_str)
sent_neg_review = sent_tokenizer1.tokenize(reviews_neg_str)
sent_review = sent_tokenizer1.tokenize(reviews_str)
sent_fullreview[:5]

['[',
 "'Worth every penny.",
 'Best value for money mid range killer.',
 'Balance of power, storage, camera and battery.',
 "', 'Pros:', '1) Superb camera."]

Extract common features using Apriori algorithm.

Apriori algorithm is used for frequent item set mining and association rule learning over transactional data. It proceeds by identifying the frequent individual items in the data and extending them to larger and larger item sets as long as those item sets appear sufficiently often in the data. The frequent item sets determined by Apriori can be used to determine association rules

Further details about it are mentioned in the research paper
**Fast Algorithms for Mining Association Rules
Rakesh Agrawal Ramakrishnan S&ant*
IBM Almaden Research Center
650 Harry Road, San Jose, CA 95120**

The python implementaion of the code for apriori alogirthm is taken from https://github.com/asaini/Apriori

In [156]:
import sys

from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser


def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
        """calculates the support for items in the itemSet and returns a subset
       of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= minSupport:
                        _itemSet.add(item)

        return _itemSet


def joinSet(itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet,
                                        transactionList,
                                        minSupport,
                                        freqSet)

    currentLSet = oneCSet
    k = 2
    while(currentLSet != set([])):
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,
                                                transactionList,
                                                minSupport,
                                                freqSet)
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
            """local function which Returns the support of an item"""
            return float(freqSet[item])/len(transactionList)

    toRetItems = []
    for key, value in list(largeSet.items()):
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])

    toRetRules = []
    for key, value in list(largeSet.items())[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item)/getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)),
                                           confidence))
    return toRetItems, toRetRules


def printResults(items):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    for item, support in sorted(items, key=lambda item_support: item_support[1], reverse=True):
        print(str(item), support)
    #print ("\n------------------------ RULES:")
    #for rule, confidence in sorted(rules, key=lambda rule_confidence: rule_confidence[1]):
        #pre, post = rule
        #print (str(pre), str(post), confidence)


Some utility methods that can extract words after checking acceptable conditions and stemming

In [155]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

stemmer = nltk.stem.porter.PorterStemmer()

def stem(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    word = word.replace("'","").replace('"','').replace('.','')
    word1 = stemmer.stem(word)
    return word1

def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool((2 <= len(word) <= 40) and word.lower() not in stopwords)
    return accepted
        
def get_terms(tree):
    term = [ stem(w) for w in tree if acceptable_word(w) ]
    yield term

We create a sets of nouns in each sentence for the apriori

In [154]:
def is_noun(n):
    if n=='NN' or n=='NNS' or n=='NNP' or n=='NNPS':
        return True

revset=[]
for line in sent_fullreview:
    a = nltk.word_tokenize(line)
    nouns = [word for (word, pos) in nltk.pos_tag(a) if is_noun(pos)] 
    terms = get_terms(nouns)
        
    for term in terms:   
        tempset=[]
        for word in term:
            tempset.append(word)
        revset.append(tempset)
print(revset[:20])

[[], ['penni'], ['valu', 'money', 'mid', 'rang', 'killer'], ['balanc', 'power', 'storag', 'camera', 'batteri'], ["'pro", 'superb', 'camera'], ['protrait', 'mode'], ['night', 'light', 'snap'], ['blaze', 'dashcharg'], ['hour', 'minut'], ['sound'], ['oxygen', 'o', 'sd835', 'gb', 'ram'], ['updat', "'con", 'glitch', 'len', 'effect', 'cam'], ['cam'], ['segment'], ['tast', "'no"], ['custom', 'support', 'oneplu'], ['phone', 'charger', 'pin', 'devic', 'point'], ['tragedi'], ['power', 'suppli', 'smoke'], ['custom', 'care', 'time', 'respons']]


Run the apriori alogrithm on the noun sets to identify the most frequent nouns. We also need to mention the minimum support and the confidence. The resulting output items are the most common features in the reviews.

In [147]:
items, rules = runApriori(revset, 0.01, 0.05)
printResults(items)

('phone',) 0.22494205797377528
('camera',) 0.09775534177858208
('batteri',) 0.041908753214591865
('perform',) 0.036828904340095885
('product',) 0.03552719306600629
('qualiti',) 0.03438422706924469
('price',) 0.03406673651458869
('day',) 0.03282852335143029
('camera', 'phone') 0.02743118392227831
('devic',) 0.02431977648664952
('issu',) 0.024192780264787123
('oneplu',) 0.024002285931993522
('featur',) 0.023875289710131125
('charg',) 0.023113312378956728
('amazon',) 0.022764072768835127
('time',) 0.02251008032511033
('mobil',) 0.02133536527288313
('money',) 0.018128710670857542
('iphon',) 0.017334984284217543
('display',) 0.016319014509318348
('mode',) 0.015398291900815951
('ram',) 0.014826808902435153
('life',) 0.014668063625107154
('experi',) 0.014636314569641554
('speed',) 0.014604565514175953
('dash',) 0.014509318347779153
('screen',) 0.014128329682191954
('phone', 'price') 0.014096580626726356
('flagship',) 0.013779090072070356
('deliveri',) 0.013620344794742357
('camera', 'qualiti'

Next we use the Liu and Hu Opinion Lexicon to classify each sentence as positive, negitive or neutral. This is a simple algorithm which counts the number of positive and negitive words in a sentence and classifies the sentence accordingly. If any negation words like no or not are present, the sentiment is reversed.

This process may take a couple of hours based on the size of your dataset

In [67]:
def custom_liu_hu_lexicon(sentence):
    '''Takes in a sentence and returns the sentiment of the sentence by counting the no of positive and negitive 
    and negitive words and by reversing the sentiment if the words NO or NOT are present
    '''
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank

    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]

    x = list(range(len(tokenized_sent))) 
    y = []
    isNegation = False
    negationWords = ['no','not','never','none','hardly','rarely','scarcely','']

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
            y.append(1) # positive
        elif word in opinion_lexicon.negative():
            neg_words += 1
            y.append(-1) # negative
        else:
            y.append(0) # neutral
            
        if word in negationWords:
            isNegation = True

    if pos_words > neg_words and isNegation==True:
        return 'neg'
    elif pos_words > neg_words:
        return 'pos'
    elif pos_words < neg_words and isNegation==True:
        return 'pos'
    elif pos_words < neg_words:
        return 'neg'
    elif pos_words == neg_words:
        return 'neutral'

neutral_review=[]
positive_review=[]
negative_review=[]
for sentence in sent_review:
    for i in items:
        if i[0][0] in sentence:
            #print(i[0][0] +"--" + sentence)
            x=custom_liu_hu_lexicon(sentence)
            if(x=="pos"):
                positive_review.append(sentence)
            elif(x=="neg"):
                negative_review.append(sentence)
            else:
                neutral_review.append(sentence)
            break

for sentence in sent_neg_review:
    for i in items:
        if i[0][0] in sentence:
            #print(i[0][0] +"--" + sentence)
            negative_review.append(sentence)
            break
print('done')            

done


In [68]:
print(positive_review[:10])

["', 'Pros:', '1) Superb camera.", 'Protrait mode is awesome (90/100).', '\', "Highly recommended to feel the premium segment\'s', 'customer support from oneplus.', 'While charging my phone, charger pin got burnt as well as the device’s charging point.', 'Or else put some more money and buy from a renowned company.', 'Don’t risk your life for good camera or fast charging.', "'My whole experience with smart phones has changed with one plus 5 , one hell of a phone , pakka value for money ,and happy for the decision", "I Must say this phone is better than i phone's", "'The better it gets,a value for money phone as compared to iPhone,but frankly I would like to tell one plus 5 = Samsung + iPhone']["]


We dump our results into a file so that we need not repeat this time consuming process again if we need those results later.

In [69]:
import pickle
with open('D:/all_pos_rating_op5.txt', 'wb') as fp:  
    pickle.dump(positive_review, fp)

print('done')    
    
with open('D:/all_neg_rating_op5.txt', 'wb') as fp:  
    pickle.dump(negative_review, fp)

print('done')

with open('D:/all_neutral_rating_op5.txt', 'wb') as fp:  
    pickle.dump(neutral_review, fp)
print('done')

done
done
done


We fetch the dumped data back into python lists

In [70]:
import pickle
pos_sentences=[];neg_sentences=[];neutral_sentences=[]
with open ('D:/all_pos_rating_op5.txt', 'rb') as fp:
    pos_sentences = pickle.load(fp)
with open ('D:/all_neg_rating_op5.txt', 'rb') as fp:
    neg_sentences = pickle.load(fp)
with open ('D:/all_neutral_rating_op5.txt', 'rb') as fp:
    neutral_sentences = pickle.load(fp)

print(str(pos_sentences[:5]))

["', 'Pros:', '1) Superb camera.", 'Protrait mode is awesome (90/100).', '\', "Highly recommended to feel the premium segment\'s', 'customer support from oneplus.', 'While charging my phone, charger pin got burnt as well as the device’s charging point.']


After classifying the reviews as positive and negitive, we tokenize those reviews into words and apply part of speech(POS) tagging on them

In [71]:
import nltk
pos_tokens=[];pos_tokens_postagged=[];neg_tokens=[];neg_tokens_postagged=[];neut_tokens=[];neut_tokens_postagged=[];
'''Tokenize the sentences into words'''
for sent in pos_sentences: 
    pos_tokens.append(nltk.word_tokenize(sent))    
for sent in neg_sentences:
    neg_tokens.append(nltk.word_tokenize(sent))    
for sent in neutral_sentences:
    neut_tokens.append(nltk.word_tokenize(sent))

'''Apply Part of speech tagging for the tokenized words'''
for sent in pos_tokens:    
    pos_tokens_postagged.append(nltk.tag.pos_tag(sent))
for sent in neg_tokens:    
    neg_tokens_postagged.append(nltk.tag.pos_tag(sent))
for sent in neut_tokens:    
    neut_tokens_postagged.append(nltk.tag.pos_tag(sent))  
    
print(pos_tokens_postagged[:5])

[[("'", "''"), (',', ','), ("'Pros", 'NNS'), (':', ':'), ("'", "''"), (',', ','), ("'1", "''"), (')', ')'), ('Superb', 'NNP'), ('camera', 'NN'), ('.', '.')], [('Protrait', 'NNP'), ('mode', 'NN'), ('is', 'VBZ'), ('awesome', 'JJ'), ('(', '('), ('90/100', 'CD'), (')', ')'), ('.', '.')], [("'", "''"), (',', ','), ('``', '``'), ('Highly', 'RB'), ('recommended', 'VBN'), ('to', 'TO'), ('feel', 'VB'), ('the', 'DT'), ('premium', 'JJ'), ('segment', 'NN'), ("'s", 'POS')], [('customer', 'NN'), ('support', 'NN'), ('from', 'IN'), ('oneplus', 'NN'), ('.', '.')], [('While', 'IN'), ('charging', 'VBG'), ('my', 'PRP$'), ('phone', 'NN'), (',', ','), ('charger', 'NN'), ('pin', 'NN'), ('got', 'VBD'), ('burnt', 'RB'), ('as', 'RB'), ('well', 'RB'), ('as', 'IN'), ('the', 'DT'), ('device', 'NN'), ('’', 'NNP'), ('s', 'VBD'), ('charging', 'VBG'), ('point', 'NN'), ('.', '.')]]


After POS tagging is done, we need to extract features that are opinion phrases using nltk chunking.

We also remove the stop words, filter out words that are too small and or that are too big, stem and lemmatize the words so that we can find the repeting phrases accurately

**This paper has been used to identify the chunking patterns:**
Su Su Htay and Khin Thidar Lynn, “Extracting Product Features and Opinion Words Using Pattern Knowledge in Customer Reviews,” The Scientific World Journal, vol. 2013, Article ID 394758, 5 pages, 2013. doi:10.1155/2013/394758

In [72]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

lem = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()
lem_word_mapping={}

gram = r"""       
    P1:{<JJ><NN|NNS>}
    P2:{<JJ><NN|NNS><NN|NNS>}
    P3:{<RB|RBR|RBS><JJ>}
    P4:{<RB|RBR|RBS><JJ|RB|RBR|RBS><NN|NNS>}
    P5:{<RB|RBR|RBS><VBN|VBD>}
    P6:{<RB|RBR|RBS><RB|RBR|RBS><JJ>}
    P7:{<VBN|VBD><NN|NNS>}
    P8:{<VBN|VBD><RB|RBR|RBS>}
"""

def leaves(tree):
    """Finds leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label() in ['P1','P2','P3','P4','P5','P6','P7','P8']):
        yield subtree.leaves()

def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    word1 = stemmer.stem(word)
    word1 = lem.lemmatize(word1)    
    if word!=word1:
        lem_word_mapping[word1]=word
    return word1

def get_terms(tree):
    """Returns the words after checking acceptable conditions, normalizing and lemmatizing"""
    term = [ normalise(w) for w in tree if acceptable_word(w) ]
    yield term
    
def get_t_norm(tree):
    """Parse leaves in chunk and return after checking acceptable conditions, normalizing and lemmatizing"""
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        yield term


In [77]:
def extractOpinionPhrases(posTaggedData):
    '''Extract noun phrases from part of speech tagged tokenized words'''
    output=[]
    for tup in posTaggedData:
        chunk = nltk.RegexpParser(gram)
        tr = chunk.parse(tup)
        term = get_t_norm(tr)

        for ter in term:    
            wordConcat=""
            for word in ter:
                if wordConcat=="":
                    #Replace good, wonderful and awesome with great
                    wordConcat = wordConcat + word.replace("good","great").replace("wonderful","great").replace("awesome","great").replace("awesom","great")
                else:
                    wordConcat = wordConcat + " " +  word
            if(len(ter)>1):
                output.append(wordConcat) 
    return output

ExtractedWords_pos = extractOpinionPhrases(pos_tokens_postagged)
ExtractedWords_neg = extractOpinionPhrases(neg_tokens_postagged)
        
print(ExtractedWords_pos[:50])

['highli recommend', 'premium segment', 'got burnt', 'el put', 'renown compani', 'great camera', 'fast charg', 'whole experi', 'smart phone', 'ever use', 'commit meet', "'sure recommend", "'excel phone", 'great pic', 'light weight', 'time deliveri', "'excel phone", 'low light', 'great phone', "'great phone..", 'range.. even', 'great featur', "'wonder camera", 'great mobil', 'best price', 'amaz featur', "'nice mobil", 'fast mobil', 'long last', 'superb camera', 'fast del', '.great job', 'realli awesom', "'excel phone", 'time.. thank', 'smart phone', 'great phone', 'great camera', 'excel featur', 'great pictur', 'nice build..veri', "'brilliant phone", 'great camera', 'super fast', 'ad attract', 'super cool', "'great product", 'smooth touch', 'great phone..', 'great perform']


After extracting the opinion words and features, now we identify the most common opinions by using frequency distribution

In [96]:
freqdist_neg = nltk.FreqDist(word for word in ExtractedWords_neg)
mc_neg = freqdist_neg.most_common()
freqdist_pos = nltk.FreqDist(word for word in ExtractedWords_pos)
mc_pos = freqdist_pos.most_common()

print(mc_pos[:50])
print("----")
print(mc_neg[:50])

[('great phone', 290), ('great camera', 168), ('great batteri', 93), ('dual camera', 90), ('great product', 78), ('great featur', 64), ('great perform', 63), ('amaz phone', 51), ('dash charg', 49), ('low light', 46), ("'excel phone", 42), ('smart phone', 40), ('great mobil', 39), ('nice phone', 39), ('realli good', 38), ('excel camera', 37), ('ever use', 36), ('great price', 36), ('build qualiti', 36), ('excel phone', 35), ('great experi', 31), ('top notch', 29), ('super fast', 28), ('great job', 26), ('high end', 26), ('great devic', 25), ('built qualiti', 24), ('pretti good', 24), ('heavi usag', 24), ('great qualiti', 23), ('mobil phone', 23), ('great look', 21), ('reason price', 21), ('first time', 21), ('android phone', 21), ('also good', 20), ('use till', 20), ('great deal', 20), ('worth buy', 19), ('great speed', 19), ('light weight', 18), ("'great product", 18), ('fast deliveri', 18), ('nice product', 18), ('great valu', 18), ('whole day', 18), ('amaz product', 18), ('realli awe

We then unlemmatize and unstem the opinion words extracted above using the dictionary we created which mapped the words raw form with the stemmed and lemmatized form

In [157]:
import inflect
p = inflect.engine()
def replacewords(mc):
    newmc=[]
    for a in mc:
        newword="";found=False;
        for b in a[0].split():            
            for x in lem_word_mapping:
                #print(x)
                #print(b)
                if b==x:
                    found=True
                    sing=(lem_word_mapping[x] if p.singular_noun(lem_word_mapping[x])==False else p.singular_noun(lem_word_mapping[x]))
                    if newword=="":
                        newword = newword + sing
                    else:
                        newword = newword + " " +  sing
            if found==False:
                if newword=="":
                    newword = newword + b
                else:
                    newword = newword + " " +  b
                    #print(newword)
        newmc.append((newword,a[1]))
    return newmc

final_neg = replacewords(mc_neg)
final_pos = replacewords(mc_pos)

print("Postive Opinon phrases:")
print(final_pos[:50])

Postive Opinon phrases:
[('greatnes phone', 290), ('greatnes camera', 168), ('greatnes battery', 93), ('dual camera', 90), ('greatnes product', 78), ('greatnes feature', 64), ('greatnes performance', 63), ('amazing phone', 51), ('dashing charge', 49), ('low light', 46), ("'excellent phone", 42), ('smart phone', 40), ('greatnes mobile', 39), ('nice phone', 39), ('really goodnes', 38), ('excellent camera', 37), ('ever using', 36), ('greatnes pricing', 36), ('building quality', 36), ('excellent phone', 35), ('greatnes experience', 31), ('top notch', 29), ('super fastnes', 28), ('greatnes job', 26), ('high end', 26), ('greatnes device', 25), ('built quality', 24), ('pretty goodnes', 24), ('heavy usage', 24), ('greatnes quality', 23), ('mobile phone', 23), ('greatnes look', 21), ('reason pricing', 21), ('first time', 21), ('android phone', 21), ('also goodnes', 20), ('using', 20), ('greatnes deal', 20), ('worthful buying', 19), ('greatnes speed', 19), ('light', 18), ("'greate product", 18),

In [158]:
print("Negitive Opinion phrases:")
print(final_neg[:50])

Negitive Opinion phrases:
[('low light', 44), ('greatnes phone', 40), ('dual camera', 31), ('new phone', 28), ('greatnes camera', 27), ('really disappointment', 24), ('many time', 18), ('greatnes battery', 17), ('light condition', 16), ('greatnes feature', 15), ('old phone', 15), ('full day', 14), ('rear camera', 13), ('smart phone', 13), ('tempered glass', 12), ('high end', 12), ('dashing charge', 12), ('many person', 12), ('defect product', 12), ('building quality', 11), ('mobile phone', 11), ('greatnes product', 11), ('also goodnes', 11), ('heavy game', 10), ('full charge', 10), ('heavy usage', 10), ('fastnes charge', 9), ('little bit', 9), ('greatnes picture', 9), ('first time', 9), ('never', 9), ('amazing phone', 8), ('greatnes job', 8), ('greatnes performance', 8), ('hardware issue', 8), ('majority issue', 8), ('next day', 8), ('electr shock', 8), ('super fastnes', 7), ('depth effect', 7), ('normal usage', 7), ('greatnes quality', 7), ('first day', 7), ('defect piece', 7), ('ever

Finally for each feature, if the feature is contained in the most common opinion phrases, we extract the opinions corresponding to that feature and opinion phrase. Only first 3 opinions for corresponding feature and first 10 corresponding opinions are fetched here as the dataset is huge. You may remove the count and the break statements if you want all the corresponding reviews

In [138]:
def finResult(itemArr, opinionPhrases, sentenceArr):
    for item,support in sorted(items, key=lambda item_support: item_support[1], reverse=True):
        count=0
        print("----------"+item[0]+"----------")
        for phrase,freq in sorted(opinionPhrases, key=lambda phrase_freq: phrase_freq[1], reverse=True): 
            pcount=0
            if normalise(item[0]) in normalise(phrase):
                count+=1
                print("---"+phrase+"---")
                for l in sentenceArr:
                    if normalise(phrase) in normalise(l):   
                        for b in zip(l.split(" ")[:-1], l.split(" ")[1:]):
                            #print(b[0]+" "+b[1])
                            if normalise(b[0])==normalise(item[0]):
                                print(l.replace("'","").replace("]","").replace("[",""))
                                pcount+=1
                                break
                            elif (normalise(b[0])+" "+normalise(b[1]))==normalise(item[0]):
                                print(l.replace("'","").replace("]","").replace("[",""))
                                pcount+=1
                                break
                        if pcount==10:
                            break                
            if count==3:
                break 
                
posSentStr = ""
posSentStr = posSentStr.join(pos_sentences)
posTokenSentences = sent_tokenizer1.tokenize(posSentStr)
finResult(items, mc_pos, posTokenSentences)

----------phone----------
---'great phone..---
probably the best phone I have everGreat phone...camera is awesome..totally loved it.
Great Phone...... Great Experience, "Im using this phone (OnePlus 5 slate grey 128gb) from last 1 month.
What an awesome phone dash charger is kickass seriously changes my charging habit camera is also pleasant phone looks decent toGreat Phone....superb performance....lightening fastPhone processor , performance are awesome.Battery life satisfactory.Amazon... Such a wonderful product... Full tohappyHappy to have Onplus 5, as this is my first OnePlusGood phone, excellent features at this price range.
---great phone..---
probably the best phone I have everGreat phone...camera is awesome..totally loved it.
Great Phone...... Great Experience, "Im using this phone (OnePlus 5 slate grey 128gb) from last 1 month.
What an awesome phone dash charger is kickass seriously changes my charging habit camera is also pleasant phone looks decent toGreat Phone....superb pe

I am just loving it, Excellent battery, awesome camera, top notch performance.When you toggle the screen or slide it to view facebook it will show u the lag at the back, 3) There is no image electronic stabilisation for the rear camera.not cribbing quality should be the same, 8) The Indian version of 1+5 is made by OPPO the same company which makes the thinnest lcd ever when crashes, 9) The OnePlus 5 also includes a one-year manufacturer warranty for device and six-month manufacturer warranty for in-box accessories including batteries from the date of purchase.Wish they consider looking into both sim cards supporting Volte, Pro Time, 1) Dash charge :- One best thing in the world is dashcharge.So go for this phone currently the best dual camera it has little issues while snapping to edges but its great belive me.Even the mono speaker for ringtone is loud enough however its delicate, 7) Customize: The more you think about customizing it the more you feel pratical at it.
, 1+5 though come

I am OnePlus die heart fan, Awesome mobile from OnePlus,Upgrated from OP3 to OP5 such a wonderful experience guyss trust me it worth every penny phone feels premium in hands, smooth OS (almost stock android experience),it is better than Iphone 7 plus Absolutely stunning camera, the portrait mode is an added advantage dash charging overall a good experience and great products from one plus
"The best android smartphone at the moment.. Just amazing on everything.. camera, battery, screen, OS.."Great product- super smooth interface and camera is a killer.Downloading speed is superb due to 8GB RAM.been a week Im using this phone", I would say .. I loved it, The only minus for the product are, No ear phones, I dint find move option in the gallery ( to transfer pic from one folder to otherThe performance speed is
Absolutely classy phone.The processing speed is mind blowing and multi tasking is lightening fast.Thank you ONE PLUS for the great product and I hope to see many more in the future.


"Wow amazing every one must buy it,really its a flagshipVery prompt next day delivery.
Wow exciting opo 5 received on next day of launch,, Now i m owning opo 1,opo 2,opo 3t,opo 5 all versions, all are awesome phones
I have paid 80 Rd Extra and Yes it was worth paying that ,Even though it was heavily raining the Courier Guy has come for the delivery on very next day of the Order.
!Got it delivered after the next day of launch thanks to excellent Amazon delivery... Battery life is excellent... Camera is good for the price ... Phone runs smooth thanks to Snapdragon 835... Its a grt price to performance phone ..."
Amazing Amazon, Awesome DhinChak Phone... 1 Plus 5....  5 Star...Ram 8GB, Space 128 Gb.. Dual Camera.. Dual Sim... Awesome Processor....  Amazing Touch and Feel.... Go For It, "Rocket Service by Amazon... Ordered At 3 pm... Next day Wifes Birthday... Received @8Am Sharp....", Go For Prime...Best Service
Hai I got one plus 5 in a next day delivery , delivery time is tooo good, Abo

---excel camera---
----------devic----------
---devic heat---
---great device.thi---
---oneplu device.i---
----------issu----------
---heat issu---
---littl issu---
---big issu---
, "6) Front speaker is awesome.So my experience is v good.However I am backing up my videos on cloud these days & deleting already watched movies so its not a big issue forworth spending 38k for this phone, On time delivery.7.5/10
"If youre ok with this price tag just go for it,", "Bought it on June 27th and till now there no big issue about the phone.
, Two big issues I am facing from day 10th onwards, those are the battery life and the Dash charge not working.
----------oneplu----------
---oneplu devic---
---oneplu phone---
---oneplu famili---
----------featur----------
---great featur---
Awesome phone with some great features and amazing battery life.is
Such an awesome mobile I have been waiting for after 1 plusawesome phone with great features and at an fantasticBest phone for the price range.
Mind blowin

...

Best android device,power is awesome and nice design
, The front camera is great though the back camera can need some improvement and also the phone feels pretty good in"I would really praise the one plus team who have put such an effort in making this humongous spec phone, For all the Android fans this is your dream come true, and for iPhone users I wont comment if they are happy with iPhone bcoz everyone has their own", This phone is my 15th Android device and I should say that I am really stunned and mesmerised by holding and operating OP5 in my hand.
", The only thing I would suggest is to order a tempered glass and a good hybrid case for protection in addition to the phone.an amazing phone with loaded feature.Just long press the focus area and the exposure gets locked and you will have a much better video.Its the best phone camera for vlogging and a new addition to my videoOne of the best Android phones I have owned.Thanks to OnePlus for providing such great spec at an affordable 

, Other features worth mentioning: Reading Mode, Gaming DND, Slider (For people who attend lot of meetings), NFC, HD Display, Audio..., "Final verdict: If you have 3T, upgrade if you are looking for camera upgrade/premium look of the phone, otherwise its worth waiting for a OPO phone with water resistance and other bezel-less design which they may release next year.
Excellent masterpiece ..its been more than a week now using this phone... amazing features.. great battery backup..dash charge is the best feature this phone has..almost 0 to 100% in an hour time..and last for almost 24 hours on heavy usage...worth for the cost...fanAlthough the midnight black is little slippery to hold, the premium look is worth it.Overall very happy with the device.
"Good phone, premium looking design.
----------phone----------
---'great phone..---
probably the best phone I have everGreat phone...camera is awesome..totally loved it.
Great Phone...... Great Experience, "Im using this phone (OnePlus 5 slate

----------batteri----------
---great batteri---
---batteri life---
---amaz batteri---
----------batteri----------
---great batteri---
---batteri life---
---amaz batteri---
----------thing----------
----------camera----------
---great camera---
Superb slick phone, great camera nice depth mode like DSLR cameraphone is a beast n crazy fast...I am very happy with the purchase...
portrail photo is the awsome feature, perfect andoid product at hisAwesome product with great camera features.. Lighting fast processor...
Nice phone with great camera battery backup
Amazing phone in good price,with great camera experience
Though I have used it just for couple of days, first impression is excellent.Slim, light weight with sharp and bright display.No heating issues as reported by few earlier.Superb camera specially in day/ bright light.Top of that we got extra exchange value for old mobile and cashback against card.Finger print scanner is so fast that sometimes I wonder if the phone even locked itse

----------month----------
---six-month manufactur---
---last month---
Bought this phone last month and so far it has been an amazing experience!!!
"I am not going to bore you with the internal specifications of this phone as everyone else is doing that.The gestures of this phone are outstanding.This sounds like a gimmick but is super useful and quick in real world day to day usage.The best feature of this camera is its ability to click RAW images.Last month OnePlus launched an update which adds Electronic Stabilisation to 4K footage as well and trust me Electronic Stabilisation(ES) is better than Optical Stabilisation(OS).Even the Pixel phones use ES and are doing very well.I am attaching the low light sample images to give you an idea about the image quality, because images in daylight look good on all smartphones these days.The phone is super snappy in day to day tasks and will definitely impress you if you play high quality games.even better - The battery takes about an hour to char

---true valu---
The OnePlus 5 is an excellent piece of craftsmanship.....great looks, awesome performance, lovely pics from camera....all in all true value for money device.
, Finally it is a good mobile and true value for money.Happy buying!!.
Great phone and a fantastic color.Performance is unquestionable and a true value for money with top end specs.
Very good phone, its a true value for
---great phone.valu---
----------video----------
---prime video---
, Mostly used app by me is Netflix and Amazon Prime Videos, watch atleast  2 or 3 seasons in day, casual apps like WhatsApp and facebook and watch some video songs", The main concern should be battery backup which is very good and Charger charges the phone like lightning.The phone felt warm.But under immense stressful condition the MSM8998 with adreno 540 screen rendering handles it well.As its a kryo architecture, successor of krait and cortex designed chips it shows promising results as comparable to what qualcomm boasted about its

In [139]:
negSentStr = ""
negSentStr = negSentStr.join(neg_sentences)
negTokenSentences = sent_tokenizer1.tokenize(negSentStr)
finResult(items, mc_neg, negTokenSentences)

----------phone----------
---bad phone..m---
, "Whenever I used the phone my ears started hurting and headache started.I switched back to my old phone if it was phone issue and worn my old phone I didnt face any issue.Recently 13 day ago I have purchase but my phone is hang 3-4 times already.. very bad phone..m very disappointed...my phone is getting hang again and again., Switch off on by ownMany a times I have to try multiple times of connection enable disable to connect toOnly thing is not good in low light cam.Will give 5 when the small issue of connectivity isI dont know whether my one plus 5 is defective or its the problem with all one plus 5. other one plus 5 user please let me know about your sound output."
"It doesnt have Bluetooth equalizer, over all i dont like the phone its same as nexusI received the phone in a white colored box..however when one plus delivers the phone ..along with the white box we get 4 back covers along with a detailed user manual and a travel bag..can 

The phone is topnotch ... No flaws .. super fast interface ... Great camera .. great performance.. great looks, though it looks somewhat like an iPhone 7plus .., A must buy for people looking for an Android device with superfast performance and flagship specsAwesome phone ❣️ best hands on experience but the ui has some bugs in it and certain games have lag issue and the camera is pixelated during the night times
Awesome device ,great camera ,cud have been cheaper ,at this price no edge screen,no waterproofing hurts ,rest isIt really hurts to invest 33k in a phone which created so much hype and to get only a dented camera.
, Spending 38k for the phone and the phone incurring a 7k cost in the next month itself was a big shock to", 3 Although there were bugs in portrait mode.A beautiful design with a great Camera DSLR effects and Snapdragon 835 octa core processor and no doubt a fastest charging, Dash Charging.
---rear camera---
", The only drawback is that the charger is quite bulky and 

---full batteri---
---batteri life---
----------perform----------
---great perform---
, Those who are complaining about the screen resolution, have baseless arguments unless you are going to use it more often with a VR headset.I was confused on whether to go for 8GB Ram phone.But now a days individuals are willing to pay 75,000 for phones, compared to that, its decent price", Features - No complaints - Great performance in third week running.
Im not impressed with camera performance.... I use honor6x before buying oneplus 5....comparing from honor6x...its not so good...but hardware is perfect and little heating problem while charging... Maybe they will fix with update... Hope so they willBut in item that I got I found heating issue near camera, when taking pictures continuously about more than 20 pics te area around camera heating increasingly.Otherwise camera hardware is quite decent but bokeh effect is also good but not like as dslr and price between 6gb and 8 GB 5000 that is I think

Could not get a better value for my money.Camera :- Just wow, I am DSLR user for photos having wide variety of lenses prime and zoom lenses I was never impressed with the photos quality of smartphone.Previous two phones had the same issue.really disheartening.I bought the OnePlus 8gb Ram 128 GB storage model.on the same date of delivery it got hung..can any body believe it.It has 8gb of ram.Also next day it restart 4 times in a day.2 time while using map it Hungs and restarted 1 time when the phone was getting charged and 4 th time it was just kept as it on the table and it restart.I have contacted Amazon customer care.They told technician will contact me.I am still trying to figure out how he will deal with this sudden restart issue.I need my phone to get replaced."
support from oneplus.While charging my phone, charger pin got burnt as well as the device’s charging point.Called and mailed customer care many times but there is no response at all.Most of the people given 5 star reviews 

, Spending 38k for the phone and the phone incurring a 7k cost in the next month itself was a big shock to", 3 Although there were bugs in portrait mode.A beautiful design with a great Camera DSLR effects and Snapdragon 835 octa core processor and no doubt a fastest charging, Dash Charging.
---rear camera---
", The only drawback is that the charger is quite bulky and the cable is a bit short.Lot of people have complained that phone is lacking QHD display.Another missing  feature which people have complained is the lack of waterproofing.Nice feel on hand and sleek look, Negative points:, One can not have a 100℅ perfect product.6 gigs of ram and sd835 make multitasking not only fast but enjoyable.Gaming is fun, not a single frame drop in games like PES2017, UFC, ASPHALT8.Dual camera is awesome, although it has no OIS but EIS of this one does equally good job.No stereo speaker, I mean, come on... It is a 33k phone and competition is tough this time with nokia 8 around with its OZO sound.I

...

---look terribl---
I had purchase one plus5 on  zero % interest offer of sbi i had taken six emi offer of sbi card.but offer is foolish they are charging 14% interest on", So, this is bit awkward looking mobile forShaky 4K videos look terrible.Doesn
", So, this is bit awkward looking mobile forShaky 4K videos look terrible.
----------phone----------
---bad phone..m---
, "Whenever I used the phone my ears started hurting and headache started.I switched back to my old phone if it was phone issue and worn my old phone I didnt face any issue.Recently 13 day ago I have purchase but my phone is hang 3-4 times already.. very bad phone..m very disappointed...my phone is getting hang again and again., Switch off on by ownMany a times I have to try multiple times of connection enable disable to connect toOnly thing is not good in low light cam.Will give 5 when the small issue of connectivity isI dont know whether my one plus 5 is defective or its the problem with all one plus 5. other one plus 5

"I had received my new one plus 5 yesterday and it is having some network issues with both wifi and cellular.one month .. camera doesnt seems  as expected for distant object.I was in the market for a 15k medium segment phone but was not ready to compromise on my requirement (Solid build quality, Great Camera, large battery w. fast charging, dual sim - no hybrid slot) by shortlisting either Nokia or Motorola/Lenovo.Did not like the Xiaomi, Oppo and Coolpad phones either - I had to move my budget up.
The phone is topnotch ... No flaws .. super fast interface ... Great camera .. great performance.. great looks, though it looks somewhat like an iPhone 7plus .., A must buy for people looking for an Android device with superfast performance and flagship specsAwesome phone ❣️ best hands on experience but the ui has some bugs in it and certain games have lag issue and the camera is pixelated during the night times
Awesome device ,great camera ,cud have been cheaper ,at this price no edge scree

---monthli secur---
---half month---
i love this phone and it since 1 & half month gone my phone is total working best, with 835 snap dragon, 8 gb ram, 128 gb rom i got 110gb for storage..., Midnight black color is giving super look for movie watching in night when no lights its look like no phone in your hand i am glad for that..., also i install lots of heavy games like nfs no limits, real racing, modern combat 5, unleashed, iron blade, yo-gi-oh dual links, asphalt 8 & many more small games after all this games installed my phone working super fine..., but i have problem with camera result there have many noise in night mode, also second camera we use for depth effect it not as i expected 3 out of 5, all over best phone compare to other heavy range phone ....
---next month---
, Spending 38k for the phone and the phone incurring a 7k cost in the next month itself was a big shock to", 3 Although there were bugs in portrait mode.A beautiful design with a great Camera DSLR effects and Sn

----------valu----------
---true valu---
, (Im a master mariner and no where related to these companies), Tech specs: Pls check amazon/gsmarena etc, If your in budget for 30k~40K and want a true value for money phone, One plus is the best bet.From app installation to switching between apps; multitasking, everything is as quick as possible.So, the ram size and the processor is not just for showing that configuration is better than other similar range or higher range phones.You would like portrait mode (mainly as it is new) but it is not that smart as of now and is not good when multiple objects are there.
One plus 5 users, i am returning the second one plus phone to amazon due to extreme slow internet connectivity issue, .One major concern found as WiFi range issue , internet speed reduce significantly if we move little far from router (even just 8~10 steps far).Compared to my existing phone on same location, it always give 70 ~80% less internet speed due to range (As a work around, I a

, Spending 38k for the phone and the phone incurring a 7k cost in the next month itself was a big shock to", 3 Although there were bugs in portrait mode.A beautiful design with a great Camera DSLR effects and Snapdragon 835 octa core processor and no doubt a fastest charging, Dash Charging.
---great batteri---
----------batteri----------
---great batteri---
---full batteri---
---batteri life---
